<a href="https://colab.research.google.com/github/ujwalshetty625/B-lore_house_price_prediction/blob/main/IOT_sensor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.base import BaseEstimator, TransformerMixin
import joblib
from collections import Counter


df = pd.read_csv("iot_sensor_dataset.csv")
print(df.head(12))


print("\nDtypes:\n", df.dtypes)
print("\nMissing values:\n", df.isnull().sum())
print("\nOutbreakRisk distribution:\n", df['OutbreakRisk'].value_counts())
print("\nUnique values for EColi_MPN:", sorted(df['EColi_MPN'].unique()))


num_cols = ["Water_pH", "Turbidity_NTU", "Chlorine_mg_L",
            "EColi_MPN", "Rainfall_mm", "AvgTemperature_C"]
for c in num_cols:
    df[c] = pd.to_numeric(df[c], errors='coerce')

if 'SampleID' in df.columns:
    dup_count = df['SampleID'].duplicated().sum()
    if dup_count > 0:
        print(f"\nFound {dup_count} duplicate SampleID rows. Dropping duplicates keeping first occurrence.")
        df = df.drop_duplicates(subset=['SampleID'])

print("\nMissing values after numeric coercion:\n", df.isnull().sum())


df['pH_deviation'] = (df['Water_pH'] - 7.0).abs()
df['chlorine_deficient'] = (df['Chlorine_mg_L'] < 0.3).astype(int)
df['turbidity_risky'] = (df['Turbidity_NTU'] > 5.0).astype(int)
df['turbidity_ecoli_interaction'] = df['Turbidity_NTU'] * df['EColi_MPN']
df['Rainfall_bin'] = pd.qcut(df['Rainfall_mm'], q=3, labels=['Low', 'Medium', 'High'])

print("\nFeature engineering completed. New columns added.")
print(df.head(12))

X = df.drop(columns=['SampleID', 'OutbreakRisk']) if 'SampleID' in df.columns else df.drop(columns=['OutbreakRisk'])
y = df['OutbreakRisk'].copy()
target_map = {'Low': 0, 'Medium': 1, 'High': 2}
y_encoded = y.map(target_map)
if y_encoded.isnull().any():
    raise ValueError("Found unknown labels in OutbreakRisk not in {Low, Medium, High}")
print("\nTarget mapping applied. Value counts:\n", y_encoded.value_counts())

# --- Step 5: IQR capper transformer ---
class IQRCapper(BaseEstimator, TransformerMixin):
    def __init__(self, factor=1.5, columns=None):
        self.factor = factor
        self.columns = columns
        self.bounds_ = {}
    def fit(self, X, y=None):
        X = pd.DataFrame(X, columns=self.columns)
        for col in self.columns:
            q1 = X[col].quantile(0.25)
            q3 = X[col].quantile(0.75)
            iqr = q3 - q1
            lower = q1 - self.factor * iqr
            upper = q3 + self.factor * iqr
            self.bounds_[col] = (lower, upper)
        return self
    def transform(self, X):
        X = pd.DataFrame(X, columns=self.columns).copy()
        for col, (lower, upper) in self.bounds_.items():
            X[col] = X[col].clip(lower=lower, upper=upper)
        return X.values

numeric_features = ["Water_pH", "Turbidity_NTU", "Chlorine_mg_L",
                    "EColi_MPN", "Rainfall_mm", "AvgTemperature_C",
                    "pH_deviation", "turbidity_ecoli_interaction"]

# --- Step 6: Custom transformer for bacterial presence (no lambda!) ---
def bacteria_to_binary(x):
    return np.where(x == "Yes", 1, 0).reshape(-1, 1)

# --- Step 7: Pipelines ---
numeric_pipeline = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("iqr_capper", IQRCapper(columns=numeric_features, factor=1.5)),
    ("scaler", StandardScaler())
])

preprocessor = ColumnTransformer(transformers=[
    ("num", numeric_pipeline, numeric_features),
    ("bacteria_bin", FunctionTransformer(bacteria_to_binary), ["BacterialPresence"]),
    ("rainfall_ohe", OneHotEncoder(handle_unknown="ignore", sparse_output=False), ["Rainfall_bin"])
], remainder='drop')

# --- Step 8: Train/test split (stratified) ---
X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.2, stratify=y_encoded, random_state=42
)
print("\nTrain/test split completed. Train shape:", X_train.shape, "Test shape:", X_test.shape)
print("Train class distribution:\n", y_train.value_counts())
print("Test class distribution:\n", y_test.value_counts())

# --- Step 9: Fit & transform ---
preprocessor.fit(X_train)
X_train_proc = preprocessor.transform(X_train)
X_test_proc = preprocessor.transform(X_test)

rainfall_ohe_categories = preprocessor.named_transformers_['rainfall_ohe'].categories_[0].tolist()
rainfall_ohe_colnames = [f"Rainfall_bin_{cat}" for cat in rainfall_ohe_categories]
processed_colnames = numeric_features + ["BacterialPresence_bin"] + rainfall_ohe_colnames

X_train_df = pd.DataFrame(X_train_proc, columns=processed_colnames, index=X_train.index)
X_test_df = pd.DataFrame(X_test_proc, columns=processed_colnames, index=X_test.index)

print("\nProcessed training data sample:")
print(X_train_df.head(12))

# --- Step 10: Optional SMOTE ---
smote_applied = False
try:
    from imblearn.over_sampling import SMOTE
    counts = Counter(y_train)
    maj = max(counts.values())
    minc = min(counts.values())
    if minc < 0.5 * maj:
        print("Significant imbalance detected, applying SMOTE on training data...")
        sm = SMOTE(random_state=42)
        X_train_res, y_train_res = sm.fit_resample(X_train_df, y_train.values)
        smote_applied = True
        print("After SMOTE:", Counter(y_train_res))
        X_train_df = pd.DataFrame(X_train_res, columns=processed_colnames)
        y_train = pd.Series(y_train_res, name='OutbreakRisk')
    else:
        print("Imbalance not severe; skipping SMOTE.")
except Exception as e:
    print("imblearn not available or SMOTE failed. Skipping SMOTE. Error:", e)

# --- Step 11: Save preprocessor ---
os.makedirs("model_artifacts", exist_ok=True)
preprocessor_file = "model_artifacts/preprocessor_pipeline.joblib"
joblib.dump(preprocessor, preprocessor_file)
print(f"\nPreprocessor pipeline saved to: {preprocessor_file}")

print("\nFinal shapes:")
print("X_train:", X_train_df.shape, "y_train:", y_train.shape)
print("X_test:", X_test_df.shape, "y_test:", y_test.shape)
print("\nProcessed feature names:\n", processed_colnames)


    SampleID  Water_pH  Turbidity_NTU  Chlorine_mg_L BacterialPresence  \
0          1      7.35           0.78           0.82                No   
1          2      6.90           0.10           0.68               Yes   
2          3      7.45           0.64           1.19               Yes   
3          4      8.07           0.18           0.06                No   
4          5      6.84           2.48           0.88                No   
5          6      6.84           4.43           0.84               Yes   
6          7      8.11           0.11           0.21                No   
7          8      7.54           4.66           0.80                No   
8          9      6.67           1.11           0.60                No   
9         10      7.38           0.53           0.43               Yes   
10        11      6.68           0.12           0.25               Yes   
11        12      6.67           1.04           0.28                No   

    EColi_MPN  Rainfall_mm  AvgTemper

In [2]:
import sklearn
sk_version = sklearn.__version__
print(sk_version)

1.6.1


In [3]:


import os
import time
import joblib
from collections import Counter

import numpy as np
import pandas as pd

# sklearn imports
from sklearn.model_selection import train_test_split, StratifiedKFold, RandomizedSearchCV
from sklearn.pipeline import Pipeline as SKPipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier, StackingClassifier
from sklearn.metrics import (accuracy_score, balanced_accuracy_score, f1_score,
                             classification_report, confusion_matrix, recall_score)
from sklearn.calibration import CalibratedClassifierCV

# Optional libs
try:
    from imblearn.over_sampling import SMOTE
    from imblearn.pipeline import Pipeline as ImbPipeline
    IMBLEARN_AVAILABLE = True
except Exception:
    IMBLEARN_AVAILABLE = False

try:
    import xgboost as xgb
    XGBOOST_AVAILABLE = True
except Exception:
    XGBOOST_AVAILABLE = False

try:
    import shap
    SHAP_AVAILABLE = True
except Exception:
    SHAP_AVAILABLE = False

# ---------- Configuration ----------
RANDOM_STATE = 42
N_JOBS = -1
ARTIFACT_DIR = "model_artifacts"
CSV_PATH = "iot_sensor_dataset.csv"  # change if needed

# ---------- Utility / Transformers (top-level classes only) ----------
class IQRCapper(BaseEstimator, TransformerMixin):
    """Caps numeric columns at [Q1 - k*IQR, Q3 + k*IQR]."""
    def __init__(self, factor=1.5, columns=None):
        self.factor = factor
        self.columns = columns  # store as given, no modification
        self.bounds_ = {}

    def fit(self, X, y=None):
        cols = self.columns
        Xdf = pd.DataFrame(X, columns=cols)
        self.bounds_ = {}
        for col in cols:
            q1 = Xdf[col].quantile(0.25)
            q3 = Xdf[col].quantile(0.75)
            iqr = q3 - q1
            lower = q1 - self.factor * iqr
            upper = q3 + self.factor * iqr
            self.bounds_[col] = (lower, upper)
        return self

    def transform(self, X):
        cols = self.columns
        Xdf = pd.DataFrame(X, columns=cols).copy()
        for col, (lower, upper) in self.bounds_.items():
            Xdf[col] = Xdf[col].clip(lower=lower, upper=upper)
        return Xdf.values


class BacteriaPresenceEncoder(BaseEstimator, TransformerMixin):
    """Encodes BacterialPresence 'Yes'/'No' as 1/0 in a picklable transformer."""
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        # X may be DataFrame/array-like with shape (n_samples, 1)
        arr = np.array(X).reshape(-1,)
        return np.where(arr == "Yes", 1, 0).reshape(-1, 1)

# ---------- Load data ----------
if not os.path.exists(CSV_PATH):
    raise FileNotFoundError(f"CSV file not found at '{CSV_PATH}'. Put your iot_sensor_dataset.csv in the same folder or update CSV_PATH.")

df = pd.read_csv(CSV_PATH)
print("Loaded CSV. shape:", df.shape)
print(df.head(8))

# ---------- Basic EDA & preprocessing-ready steps ----------
num_cols = ["Water_pH", "Turbidity_NTU", "Chlorine_mg_L", "EColi_MPN", "Rainfall_mm", "AvgTemperature_C"]
for c in num_cols:
    df[c] = pd.to_numeric(df[c], errors='coerce')

# Feature engineering
df['pH_deviation'] = (df['Water_pH'] - 7.0).abs()
df['chlorine_deficient'] = (df['Chlorine_mg_L'] < 0.3).astype(int)
df['turbidity_risky'] = (df['Turbidity_NTU'] > 5.0).astype(int)
df['turbidity_ecoli_interaction'] = df['Turbidity_NTU'] * df['EColi_MPN']
# Rainfall bins (3 quantile bins)
df['Rainfall_bin'] = pd.qcut(df['Rainfall_mm'], q=3, labels=['Low', 'Medium', 'High'])

# Target mapping
target_map = {'Low': 0, 'Medium': 1, 'High': 2}
if 'OutbreakRisk' not in df.columns:
    raise KeyError("OutbreakRisk column not found in CSV.")
y = df['OutbreakRisk'].map(target_map)
if y.isnull().any():
    raise ValueError("Found unexpected labels in OutbreakRisk. Expect only 'Low','Medium','High'.")

# Features
if 'SampleID' in df.columns:
    X = df.drop(columns=['SampleID', 'OutbreakRisk'])
else:
    X = df.drop(columns=['OutbreakRisk'])

print("\nAfter feature engineering sample:")
print(X.head(6))

# ---------- Preprocessor ----------
numeric_features = ["Water_pH", "Turbidity_NTU", "Chlorine_mg_L",
                    "EColi_MPN", "Rainfall_mm", "AvgTemperature_C",
                    "pH_deviation", "turbidity_ecoli_interaction"]

# numeric pipeline
numeric_pipeline = SKPipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("iqr_capper", IQRCapper(columns=numeric_features, factor=1.5)),
    ("scaler", StandardScaler())
])

# OneHotEncoder compatibility
from sklearn import __version__ as sklearn_version
try:
    # prefer sparse_output for modern sklearn
    ohe = OneHotEncoder(handle_unknown="ignore", sparse_output=False)
except TypeError:
    # fallback to older sklearn versions
    ohe = OneHotEncoder(handle_unknown="ignore", sparse=False)

preprocessor = ColumnTransformer(transformers=[
    ("num", numeric_pipeline, numeric_features),
    ("bacteria_bin", BacteriaPresenceEncoder(), ["BacterialPresence"]),
    ("rainfall_ohe", ohe, ["Rainfall_bin"])
], remainder='drop')

# ---------- Train/test split ----------
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
                                                    stratify=y, random_state=RANDOM_STATE)
print("\nTrain/test shapes:", X_train.shape, X_test.shape)
print("Train class distribution:", Counter(y_train))

# ---------- Fit preprocessor and create DataFrames of processed features (for inspection) ----------
preprocessor.fit(X_train)
X_train_proc = preprocessor.transform(X_train)
X_test_proc = preprocessor.transform(X_test)

# Build processed column names
rainfall_categories = preprocessor.named_transformers_['rainfall_ohe'].categories_[0].tolist()
rainfall_cols = [f"Rainfall_bin_{c}" for c in rainfall_categories]
processed_colnames = numeric_features + ["BacterialPresence_bin"] + rainfall_cols

X_train_df = pd.DataFrame(X_train_proc, columns=processed_colnames, index=X_train.index)
X_test_df = pd.DataFrame(X_test_proc, columns=processed_colnames, index=X_test.index)

print("\nProcessed features sample:")
print(X_train_df.head(6))

# ---------- Helper: Build pipeline (use SMOTE inside if available and requested) ----------
def build_pipeline(clf, use_smote=False):
    if IMBLEARN_AVAILABLE and use_smote:
        return ImbPipeline(steps=[
            ("preproc", preprocessor),
            ("smote", SMOTE(random_state=RANDOM_STATE)),
            ("clf", clf)
        ])
    else:
        return SKPipeline(steps=[
            ("preproc", preprocessor),
            ("clf", clf)
        ])

# ---------- Candidate models & hyperparameter search specs ----------
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)
models_to_search = {}

# Logistic Regression (regularized)
lr = LogisticRegression(solver='saga', class_weight='balanced', max_iter=5000, random_state=RANDOM_STATE, n_jobs=1)
models_to_search['logistic'] = {
    "pipeline": build_pipeline(lr, use_smote=False),
    "params": {
        "clf__C": [1e-3, 1e-2, 1e-1, 1, 10, 100],
        "clf__penalty": ["l2"]
    },
    "n_iter": 8
}

# Random Forest
rf = RandomForestClassifier(random_state=RANDOM_STATE, n_jobs=N_JOBS, class_weight='balanced')
models_to_search['random_forest'] = {
    "pipeline": build_pipeline(rf, use_smote=True if IMBLEARN_AVAILABLE else False),
    "params": {
        "clf__n_estimators": [100, 200, 400],
        "clf__max_depth": [None, 8, 16, 24],
        "clf__min_samples_split": [2, 5, 10],
        "clf__min_samples_leaf": [1, 2, 4]
    },
    "n_iter": 20
}


hgb = HistGradientBoostingClassifier(random_state=RANDOM_STATE)
models_to_search['hist_gb'] = {
    "pipeline": build_pipeline(hgb, use_smote=False),
    "params": {
        "clf__learning_rate": [0.01, 0.05, 0.1],
        "clf__max_iter": [100, 200, 400],
        "clf__max_depth": [None, 5, 10],
        "clf__l2_regularization": [0.0, 0.1, 1.0]
    },
    "n_iter": 18
}


if XGBOOST_AVAILABLE:
    xgb_clf = xgb.XGBClassifier(objective='multi:softprob', eval_metric='mlogloss',
                                use_label_encoder=False, n_jobs=N_JOBS, random_state=RANDOM_STATE)
    models_to_search['xgboost'] = {
        "pipeline": build_pipeline(xgb_clf, use_smote=False),
        "params": {
            "clf__n_estimators": [100, 200, 400],
            "clf__max_depth": [3, 6, 10],
            "clf__learning_rate": [0.01, 0.05, 0.1],
            "clf__subsample": [0.6, 0.8, 1.0],
            "clf__colsample_bytree": [0.6, 0.8, 1.0]
        },
        "n_iter": 25
    }


results = {}
start_time = time.time()
for name, spec in models_to_search.items():
    print(f"\n--- Tuning {name} ---")
    pipeline = spec["pipeline"]
    param_dist = spec["params"]
    n_iter = spec.get("n_iter", 20)
    # If the param grid size < n_iter, sklearn will warn; it's okay - RandomizedSearchCV handles it.
    search = RandomizedSearchCV(pipeline, param_distributions=param_dist,
                                n_iter=n_iter, scoring='f1_macro', cv=cv,
                                random_state=RANDOM_STATE, n_jobs=N_JOBS, verbose=1)
    search.fit(X_train, y_train)
    print(f"Best CV {name} score (f1_macro): {search.best_score_:.4f}")
    print("Best params:", search.best_params_)
    results[name] = search
elapsed = time.time() - start_time
print(f"\nHyperparameter tuning finished in {elapsed/60:.2f} minutes")


def evaluate_model(model_pipeline, X_test, y_test, model_name="model"):
    y_pred = model_pipeline.predict(X_test)
    try:
        y_prob = model_pipeline.predict_proba(X_test)
    except Exception:
        y_prob = None
    print(f"\n=== Evaluation: {model_name} ===")
    print("Accuracy:", accuracy_score(y_test, y_pred))
    print("Balanced accuracy:", balanced_accuracy_score(y_test, y_pred))
    print("Macro F1:", f1_score(y_test, y_pred, average='macro'))
    # recall per class; index 2 is 'High'
    recs = recall_score(y_test, y_pred, average=None)
    rec_high = recs[2] if len(recs) > 2 else None
    print("Recall for High class (label=2):", rec_high)
    print("\nClassification report:\n", classification_report(y_test, y_pred, digits=4))
    print("\nConfusion matrix:\n", confusion_matrix(y_test, y_pred))
    return {"y_pred": y_pred, "y_prob": y_prob}


eval_summary = {}
for name, search in results.items():
    best_pipe = search.best_estimator_
    eval_summary[name] = evaluate_model(best_pipe, X_test, y_test, model_name=name)

best_name = None
best_f1 = -1
for name, out in eval_summary.items():
    f1 = f1_score(y_test, out["y_pred"], average='macro')
    if f1 > best_f1:
        best_f1 = f1
        best_name = name
print(f"\nSelected best model: {best_name} with test macro-F1 = {best_f1:.4f}")

best_search = results[best_name]
best_model_pipeline = best_search.best_estimator_

# ---------- Calibrate probabilities (useful in real-world risk scoring) ----------
print("\nCalibrating best model probabilities (CalibratedClassifierCV, method='sigmoid') ...")
calibrator = CalibratedClassifierCV(best_model_pipeline, cv=5, method='sigmoid')
calibrator.fit(X_train, y_train)
print("Calibration complete.")

# ---------- Save artifacts (preprocessor, best model, calibrated) ----------
os.makedirs(ARTIFACT_DIR, exist_ok=True)
preprocessor_path = os.path.join(ARTIFACT_DIR, "preprocessor.joblib")
best_model_path = os.path.join(ARTIFACT_DIR, f"best_model_{best_name}.joblib")
calibrator_path = os.path.join(ARTIFACT_DIR, f"calibrated_{best_name}.joblib")

joblib.dump(preprocessor, preprocessor_path)
joblib.dump(best_model_pipeline, best_model_path)
joblib.dump(calibrator, calibrator_path)
print("\nSaved preprocessor, best model pipeline, and calibrated model to:", ARTIFACT_DIR)

# ---------- SHAP explainability (if available and model supports) ----------
if SHAP_AVAILABLE:
    try:
        print("\nComputing SHAP importances (this may take a bit)...")
        # Use a subset for speed & to avoid rendering in headless env
        X_sample = X_test.sample(n=min(200, len(X_test)), random_state=RANDOM_STATE)
        # Transform to model input
        X_sample_trans = best_model_pipeline.named_steps['preproc'].transform(X_sample) \
                         if hasattr(best_model_pipeline, 'named_steps') else preprocessor.transform(X_sample)
        clf = best_model_pipeline.named_steps['clf'] if hasattr(best_model_pipeline, 'named_steps') else best_model_pipeline
        # Use TreeExplainer for tree models, otherwise use a model-agnostic Explainer
        if ('XGB' in str(type(clf))) or ('RandomForest' in str(type(clf))) or ('HistGradientBoosting' in str(type(clf))):
            expl = shap.Explainer(clf)
            shap_vals = expl(X_sample_trans)
            # Save mean absolute SHAP per feature (safe alternative to plotting)
            mean_abs_shap = np.abs(shap_vals.values).mean(axis=0)
            feat_names = processed_colnames
            shap_imp_df = pd.DataFrame({"feature": feat_names, "mean_abs_shap": mean_abs_shap})
            shap_imp_df = shap_imp_df.sort_values("mean_abs_shap", ascending=False)
            shap_imp_df.to_csv(os.path.join(ARTIFACT_DIR, "shap_feature_importance.csv"), index=False)
            print("Saved SHAP feature importance csv to model_artifacts/shap_feature_importance.csv")
        else:
            # fallback: use shap.Explainer model-agnostic (may be slower)
            expl = shap.Explainer(best_model_pipeline.predict_proba, X_sample_trans)
            shap_vals = expl(X_sample_trans)
            mean_abs_shap = np.abs(shap_vals.values).mean(axis=0)
            feat_names = processed_colnames
            shap_imp_df = pd.DataFrame({"feature": feat_names, "mean_abs_shap": mean_abs_shap})
            shap_imp_df = shap_imp_df.sort_values("mean_abs_shap", ascending=False)
            shap_imp_df.to_csv(os.path.join(ARTIFACT_DIR, "shap_feature_importance.csv"), index=False)
            print("Saved model-agnostic SHAP importance to csv.")
    except Exception as e:
        print("SHAP step failed (non-fatal):", e)
else:
    print("\nSHAP not installed. To enable explainability, install shap (`pip install shap`) and re-run.")

# ---------- Optional stacking ensemble (safe saving with fallback) ----------
try:
    print("\nBuilding stacking ensemble from top tuned pipelines (if at least 2 models tuned)...")
    # sort by CV best_score_ to pick top 3
    ranked = sorted(results.items(), key=lambda kv: kv[1].best_score_, reverse=True)
    top = ranked[:3]
    if len(top) >= 2:
        estimators = []
        for name, search in top:
            estimators.append((name, search.best_estimator_))
        # final estimator simple logistic regression
        stack = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression(max_iter=2000),
                                   cv=cv, n_jobs=N_JOBS, passthrough=False)
        stack.fit(X_train, y_train)
        evaluate_model(stack, X_test, y_test, model_name="stacked_ensemble")
        # Try saving; if pickling fails, save components separately
        stack_path = os.path.join(ARTIFACT_DIR, "stacked_ensemble.joblib")
        try:
            joblib.dump(stack, stack_path)
            print("Saved stacking classifier to", stack_path)
        except Exception as e_save:
            print("Failed to pickle stacking classifier directly:", e_save)
            # fallback: save individual estimators and final_estimator separately
            fallback_dir = os.path.join(ARTIFACT_DIR, "stacking_fallback")
            os.makedirs(fallback_dir, exist_ok=True)
            joblib.dump(stack.named_estimators_, os.path.join(fallback_dir, "named_estimators.joblib"))
            joblib.dump(stack.final_estimator_, os.path.join(fallback_dir, "final_estimator.joblib"))
            print("Saved stacking components separately to", fallback_dir)
    else:
        print("Not enough tuned models to build a stacking ensemble.")
except Exception as e:
    print("Stacking step failed (non-fatal):", e)

print("\nAll done. Check the", ARTIFACT_DIR, "folder for saved models and artifacts.")


Loaded CSV. shape: (2000, 9)
   SampleID  Water_pH  Turbidity_NTU  Chlorine_mg_L BacterialPresence  \
0         1      7.35           0.78           0.82                No   
1         2      6.90           0.10           0.68               Yes   
2         3      7.45           0.64           1.19               Yes   
3         4      8.07           0.18           0.06                No   
4         5      6.84           2.48           0.88                No   
5         6      6.84           4.43           0.84               Yes   
6         7      8.11           0.11           0.21                No   
7         8      7.54           4.66           0.80                No   

   EColi_MPN  Rainfall_mm  AvgTemperature_C OutbreakRisk  
0          0        18.44              30.3          Low  
1          5        30.83              22.4         High  
2          2        26.78              32.7         High  
3          0        61.65              38.7          Low  
4          2      

/usr/local/lib/python3.12/dist-packages/sklearn/model_selection/_search.py:317: UserWarning: The total space of parameters 6 is smaller than n_iter=8. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best CV logistic score (f1_macro): 0.9311
Best params: {'clf__penalty': 'l2', 'clf__C': 100}

--- Tuning random_forest ---
Fitting 5 folds for each of 20 candidates, totalling 100 fits
Best CV random_forest score (f1_macro): 0.9598
Best params: {'clf__n_estimators': 100, 'clf__min_samples_split': 2, 'clf__min_samples_leaf': 1, 'clf__max_depth': 24}

--- Tuning hist_gb ---
Fitting 5 folds for each of 18 candidates, totalling 90 fits
Best CV hist_gb score (f1_macro): 0.9680
Best params: {'clf__max_iter': 100, 'clf__max_depth': 5, 'clf__learning_rate': 0.05, 'clf__l2_regularization': 0.0}

Hyperparameter tuning finished in 0.70 minutes

=== Evaluation: logistic ===
Accuracy: 0.9275
Balanced accuracy: 0.9493520232742662
Macro F1: 0.9182259182259184
Recall for High class (label=2): 0.9736842105263158

Classification report:
               precision    recall  f1-score   support

           0     0.7903    1.0000    0.8829        49
           1     0.9775    0.8744    0.9231       199
     

In [7]:
import pandas as pd
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np


class FeatureEngineer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()
        # Add engineered features
        X["pH_deviation"] = (X["Water_pH"] - 7.0).abs()
        X["turbidity_ecoli_interaction"] = X["Turbidity_NTU"] * np.log1p(X["EColi_MPN"])
        X["Rainfall_bin"] = pd.cut(
            X["Rainfall_mm"],
            bins=[-np.inf, 20, 50, 100, np.inf],
            labels=["Low", "Moderate", "High", "Extreme"]
        )
        return X


numeric_features = [
    "Water_pH", "Turbidity_NTU", "Chlorine_mg_L",
    "EColi_MPN", "Rainfall_mm", "AvgTemperature_C",
    "pH_deviation", "turbidity_ecoli_interaction"
]
categorical_features = ["BacterialPresence", "Rainfall_bin"]


preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), numeric_features),
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_features)
    ]
)


final_pipeline = Pipeline(steps=[
    ("features", FeatureEngineer()),
    ("preprocess", preprocessor),
    ("model", HistGradientBoostingClassifier(random_state=42))
])

X = df.drop(columns=["OutbreakRisk"])
y = df["OutbreakRisk"]
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

final_pipeline.fit(X_train, y_train)


y_pred = final_pipeline.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


joblib.dump(final_pipeline, "model_artifacts/final_pipeline.joblib")
print("✅ Final pipeline saved as model_artifacts/final_pipeline.joblib")


Accuracy: 0.9975
              precision    recall  f1-score   support

        High       1.00      0.99      1.00       152
         Low       1.00      1.00      1.00        49
      Medium       0.99      1.00      1.00       199

    accuracy                           1.00       400
   macro avg       1.00      1.00      1.00       400
weighted avg       1.00      1.00      1.00       400

✅ Final pipeline saved as model_artifacts/final_pipeline.joblib


In [20]:
import joblib
import pandas as pd

pipeline = joblib.load("model_artifacts/final_pipeline.joblib")

sample = pd.DataFrame([{
    "Water_pH": 6.5,
    "Turbidity_NTU": 4.0,
    "Chlorine_mg_L": 0.3,
    "EColi_MPN": 40,
    "Rainfall_mm": 55.0,
    "AvgTemperature_C": 30,
    "BacterialPresence": "No"
}])


pred_class = pipeline.predict(sample)[0]
pred_prob = pipeline.predict_proba(sample)[0]

print("Predicted class:", pred_class)
print("Probabilities (Low, Medium, High):", pred_prob)


Predicted class: Medium
Probabilities (Low, Medium, High): [1.29775353e-07 2.28451944e-08 9.99999847e-01]
